In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from pathlib import Path
import time

In [6]:
# Selenium options required to create a 'headless' browser
options = Options()
options.add_argument("--blink-settings=imagesEnabled=false")
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.37")

driver = webdriver.Chrome(options=options)

In [7]:
# Load ridership page from BMRCL website
driver.get("https://english.bmrc.co.in/ridership/")

# Click on Kannada toggle button to load English results
time.sleep(10)  # Allow extra time for JavaScript to load translated data
result = driver.find_element(By.CLASS_NAME, "link.top-navcustom-text").click()

In [8]:
# Results are published with a lag of about one day. 
# So get the date on the page rather than date.today()
record_date = driver.find_element(By.TAG_NAME, "h3").text
record_date

'Passenger Flow as on 08-12-2024'

In [9]:
# Initialize dict to store ridership data
day_record = {}
day_record['Record Date'] = [record_date.split()[-1]]  # Extracting date part
day_record

{'Record Date': ['08-12-2024']}

In [10]:
# Parse html for remaining data points and store in pandas dataframe
data_points = driver.find_elements(By.CLASS_NAME, "features-card.achivement-area.bg-color")

for l1 in data_points:
    for l2 in l1.text.split('\n'):
        data = l2.split(': ')
        day_record[data[0]] = [int(data[1])]

driver.quit()

day_record = pd.DataFrame(day_record)
day_record

,Record Date,Total Smart Cards,Stored Value Card,One Day Pass,Three Day Pass,Five Day Pass,Tokens,Total NCMC,Group Ticket,Total QR,QR NammaMetro,QR WhatsApp,QR Paytm
0,08-12-2024,183305,182088,1108,50,59,290250,4875,590,238955,63957,131212,43786


In [11]:
# Store data in csv file - create file if necessary
filename = "NammaMetro_Ridership_Dataset.csv"  
filePath = Path(filename)

if filePath.exists() and filePath.is_file():
    day_record.to_csv(filename, mode='a', header=False)
    print('Appended '+filename)
else:
    day_record.to_csv(filename, mode='w', header=True)
    print('Created '+filename)

Appended NammaMetro_Ridership_Dataset.csv


In [12]:
# Optimize dataset by removing duplicates and rewrite to file
df = pd.read_csv(filename, index_col=0).drop_duplicates(keep='last', ignore_index=True)
df[df.loc[:, 'Total Smart Cards':].columns] = df[df.loc[:, 'Total Smart Cards':].columns].astype('Int64')
df.to_csv(filename, mode='w', header=True)
df.tail()

,Record Date,Total Smart Cards,Stored Value Card,One Day Pass,Three Day Pass,Five Day Pass,Tokens,Total NCMC,Group Ticket,Total QR,QR NammaMetro,QR WhatsApp,QR Paytm
33,04-12-2024,491111,490387,413,200,111,205660,13014,896,190794,54771,97885,38138
34,05-12-2024,478727,477989,458,79,201,211363,12739,443,197958,56163,102339,39456
35,06-12-2024,415499,414771,411,93,224,225172,12081,219,117316,1081,102925,46269
36,07-12-2024,345932,344657,1100,80,95,268704,8885,852,258927,69323,140034,49570
37,08-12-2024,183305,182088,1108,50,59,290250,4875,590,238955,63957,131212,43786
